<hr>

# Clustering Mnist using AutoEncoder

In this notebook we will explore:

1. Building of a autoencoder architecture.
2. Training.
3. Encoder Prediction.
4. Kmeans
5. NMI metrics

__Source__: https://www.analyticsvidhya.com/blog/2018/06/unsupervised-deep-learning-computer-vision/

# Import Libraries

In [1]:
from keras.datasets import mnist

In [3]:
%pylab inline
import os
import keras
import numpy as np
import pandas as pd
import keras.backend as K

from time import time
from sklearn.cluster import KMeans
from keras import callbacks
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Dense, Input
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Load DataSet

In [7]:
(train_x, train_y), (val_x, val_y) = mnist.load_data()

# Data Normalization

In [8]:
train_x = train_x/255.
val_x = val_x/255.

train_x = train_x.reshape(-1, 784)
val_x = val_x.reshape(-1, 784)

# Model

In [9]:
# this is our input placeholder
input_img = Input(shape=(784,))

# "encoded" is the encoded representation of the input
encoded = Dense(2000, activation='relu')(input_img)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(10, activation='sigmoid')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(500, activation='relu')(encoded)
decoded = Dense(500, activation='relu')(decoded)
decoded = Dense(2000, activation='relu')(decoded)
decoded = Dense(784)(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [10]:
autoencoder.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 2000)              1570000   
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
_________________________________________________________________
dense_4 (Dense)              (None, 500)               5500      
_________________________________________________________________
dense_5 (Dense)              (None, 500)              

In [15]:
encoder = Model(input_img, encoded)

# Training

In [11]:
autoencoder.compile(optimizer='adam', loss='mse')

In [12]:
estop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

In [13]:
train_history = autoencoder.fit(train_x, train_x, epochs=500, batch_size=2048, validation_data=(val_x, val_x), callbacks=[estop])

Epoch 1/500
30/30 [==============================] - 1s 30ms/step - loss: 0.0714 - val_loss: 0.0650
Epoch 2/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0637 - val_loss: 0.0607
Epoch 3/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0563 - val_loss: 0.0500
Epoch 4/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0468 - val_loss: 0.0433
Epoch 5/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0398 - val_loss: 0.0361
Epoch 6/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0341 - val_loss: 0.0325
Epoch 7/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0309 - val_loss: 0.0293
Epoch 8/500
30/30 [==============================] - 1s 22ms/step - loss: 0.0286 - val_loss: 0.0273
Epoch 9/500
30/30 [==============================] - 1s 22ms/step - loss: 0.0266 - val_loss: 0.0252
Epoch 10/500
30/30 [==============================] - 1s 23ms/step - loss: 0.0250 - val_loss: 0.0242

# Prediction

In [16]:
pred_auto_train = encoder.predict(train_x)
pred_auto_val = encoder.predict(val_x)

# Kmeans

In [21]:
kmeans = KMeans(n_jobs=-1, n_clusters=10, n_init=20)
kmeans.fit(pred_auto_train)
pred_val = kmeans.predict(pred_auto_val)
pred_train = kmeans.predict(pred_auto_train)

In [23]:
metrics_NMI_val=normalized_mutual_info_score(val_y, pred_val)
metrics_NMI_train=normalized_mutual_info_score(train_y, pred_train)
print("Validation NMI metrics: ", metrics_NMI_val)
print("Train NMI metrics: ", metrics_NMI_train)

Validation NMI metrics:  0.75016492936774
Train NMI metrics:  0.7488622649367099
